In [17]:
import requests
from datetime import datetime
import base64

# Make a GET request to the GitHub API to get all repositories from the user "lucidrains"
page = 1
response = requests.get(f"https://api.github.com/users/lucidrains/repos?per_page=100&page={page}")

# Extract the JSON data from the response
repos = response.json()

# Create a list of records with the desired keys
records = []
while len(repos) > 0:
    for repo in repos:
        created_at = datetime.strptime(repo["created_at"], "%Y-%m-%dT%H:%M:%SZ")
        record = {
            "repository-name": repo["name"],
            "year": created_at.year,
            "month": created_at.month,
            "day": created_at.day,
            "url": repo["html_url"],
            "description": repo["description"],
        }
        # readme_response = requests.get(f"{repo['url']}/readme")
        # if readme_response.status_code == 200:
        #     readme_content = readme_response.json()["content"]
        #     record["readme-content"] = base64.b64decode(readme_content).decode("utf-8")
        # else:
        #     record["readme-content"] = "README not found"
        records.append(record)
    page += 1
    response = requests.get(f"https://api.github.com/users/lucidrains/repos?per_page=100&page={page}")
    repos = response.json()

# Print the list of records
len(records)

251

In [23]:
# Sort the records based on year, month, and day (from old to new)
sorted_records = sorted(records, key=lambda x: (x["year"], x["month"], x["day"]), reverse=True)

# Create the markdown table
table = "| Repository Name | Date | Description | Projects |\n| --- | --- | --- | --- |\n"
for record in sorted_records:
    date_string = datetime(record["year"], record["month"], record["day"]).strftime("%d %B, %Y").lstrip("0").replace(",th", ",")
    date_suffix = "th" if 11 <= record["day"] <= 13 else {1: "st", 2: "nd", 3: "rd"}.get(record["day"] % 10, "th")
    date_string = date_string.replace(" ", f"{date_suffix} ")
    repository_name = f"[{record['repository-name']}]({record['url']})"
    table += f"| {repository_name} | {date_string} | {record['description']} ||\n"

# Print the markdown table
file_path = "README.md"

prepend = '''
# :rainbow: lucidrains-projects

<p align='center'>
  <a href="https://discord.gg/xBPBXfcFHd"><img alt="Join us on Discord" src="https://img.shields.io/discord/823813159592001537?color=5865F2&logo=discord&logoColor=white"></a>
<p>

This repository gives an overview of the awesome projects created by [lucidrains](https://github.com/lucidrains) that we as LAION want to share with the community
in order to help people train new exciting models and do research with SOTA ML code. 

The whole LAION community started with crawling@home that became LAION-400M and later evolved into LAION-5B and at the same time lucidrains' awesome repository [DALLE-pytorch](https://github.com/lucidrains/DALLE-pytorch),
a replication of OpenAI's Dall-E model, that became more and more popular as we trained on CC-3m and CC-12m datasets and later on LAION-400M.

We are very thankful for the great work of lucidrains!

## Projects

'''

with open(file_path, "w") as f:
    f.write(prepend)
    f.write(table)